In [ ]:
import duckdb
import pandas as pd
from io import StringIO

# Connect to the DuckDB database
conn = duckdb.connect(':memory:')

In [21]:
df = pd.read_parquet("/home/anadirov/Documents/si.parquet")


In [25]:
def render_df_info(df: pd.DataFrame) -> str:
    buff = StringIO()
    df.info(buf=buff)
    info_str = buff.getvalue()
    lines = info_str.split("\n")
    shape = f"#### Rows{df.shape[0]}, Columns: {df.shape[1]}"
    column_lines = lines[5:27]
    markdown_table = "| # | Column | Non-Null Count | Dtype |\n"
    markdown_table += "|---|--------|----------------|-------|\n"

    # Iterate over the column lines and extract the details
    for line in column_lines:
        parts = line.split()
        index = parts[0]
        column = " ".join(parts[1:-3])
        non_null_count = parts[-3]
        dtype = parts[-1]
        
        # Add the row to the markdown table
        markdown_table += f"| {index} | {column} | {non_null_count} | {dtype} |\n"

    return f"{shape}\n{markdown_table}"
